### 1. Download Tickets Data

In [ ]:
#Download file using wget
!wget https://privdatastorage.blob.core.windows.net/github/support-tickets-classification/datasets/all_tickets.csv --quiet

In [ ]:
#Check if the file is available
!ls -l

### 2. Tickets Data Exploration

Load tickets data as dataframe

In [ ]:
import pandas as pd
import numpy as np
import textwrap

In [ ]:
# read file into pandas using a relative path. Please change the path as needed
tix_df = pd.read_table('all_tickets.csv', sep=',')

In [ ]:
#Total number of tickets
tix_df.shape

In [ ]:
#Columns in the dataframe
tix_df.columns

In [ ]:
#Check the contents of dataframe
tix_df.sample(n=5)

Checking ticket content

In [ ]:
my_wrap = textwrap.TextWrapper()

In [ ]:
tix_num = np.random.randint(0, tix_df.shape[0])
print('Title: ')
print(tix_df.loc[tix_num, 'title'])
print('Body: ')
for line in my_wrap.wrap(tix_df.loc[tix_num, 'body']):
    print(line)

In [ ]:
#Checkout missing values
tix_df.isnull().sum()

Visualization

In [ ]:
#Ticket type
tix_df['ticket_type'].value_counts().plot(kind='barh')

In [ ]:
#Category
tix_df['category'].value_counts().plot(kind='barh')

In [ ]:
#Impact
tix_df['impact'].value_counts().plot(kind='barh')

In [ ]:
#Urgency
tix_df['urgency'].value_counts().plot(kind='barh')

In [ ]:
#Sub-category 1
tix_df['sub_category1'].value_counts().plot(kind='barh', figsize=(20,15))

In [ ]:
#Sub-category 1 for specific category
tix_df[tix_df['category'].isin(['4'])]['sub_category1'].value_counts().plot(kind='barh')

### 3. Create Training & Test Dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Build a new Columns which combines 'title' and 'body'
tix_df['title_body'] = tix_df['title'].astype('str') + tix_df['body']

#Column to predict
column_to_predict = 'ticket_type'

In [ ]:
# split X and y into training and testing sets
trainX, testX, trainY, testY = train_test_split(tix_df['title_body'], tix_df[column_to_predict], random_state=2)

In [ ]:
#Traing data
print(trainX.shape)
print(trainY.shape)

In [ ]:
#Test Data
print(testX.shape)
print(testY.shape)

### 4. Tokenization & Vectorization

Using **CountVectorizer**, to get numeric features.

In [ ]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()

In [ ]:
#Feed SMS data to CountVectorizer
cvect.fit(trainX)

#Check the vocablury size
len(cvect.vocabulary_)

In [ ]:
#What is there in the vocabulary
cvect.get_feature_names()

#### How do we reduce Vocabulary?

1. Stemming / Lemmatization
2. Remove Stop words
3. Remove words who appear very rarely (min_df)
4. Remove words which appear a lot (max_df)

In [ ]:
#Stemming
from nltk.stem import PorterStemmer

Build a new CountVectorizer with stemmer

In [ ]:
analyzer = CountVectorizer().build_analyzer()
stemmer = PorterStemmer()

In [ ]:
#Function to Stem words
def get_stemmed_text(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

In [ ]:
stem_cvectorizer = CountVectorizer(analyzer=get_stemmed_text)

In [ ]:
#This will take few seconfs
stem_cvectorizer.fit(trainX)

#Vocabulary
len(stem_cvectorizer.get_feature_names())

With stemmer and minimum frequency

In [ ]:
stem_min_cvectorizer = CountVectorizer(analyzer=get_stemmed_text, min_df=5)

In [ ]:
#This will take few seconfs
stem_min_cvectorizer.fit(trainX)

#Vocabulary
len(stem_min_cvectorizer.get_feature_names())

In [ ]:
#stem_min_cvectorizer.vocabulary_

Build Document-term Matrix (DTM)

In [ ]:
#Convert Training text into Count Vectors
trainX_ct = stem_min_cvectorizer.transform(trainX)

In [ ]:
#Size of Document Term Matrix
trainX_ct.shape

In [ ]:
#Let's check the first record
trainX_ct[0]

In [ ]:
#What's there in sparse matrix
print(trainX_ct[0])

From the [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction):

> As most documents will typically use a very small subset of the words used in the corpus, the resulting matrix will have **many feature values that are zeros** (typically more than 99% of them).

> For instance, a collection of 10,000 short text documents (such as emails) will use a vocabulary with a size in the order of 100,000 unique words in total while each document will use 100 to 1000 unique words individually.

> In order to be able to **store such a matrix in memory** but also to **speed up operations**, implementations will typically use a **sparse representation** such as the implementations available in the `scipy.sparse` package.

Convert Test text also in numerical features

In [ ]:
testX_ct = stem_min_cvectorizer.transform(testX)

In [ ]:
testX_ct.shape

### 4. Building a Ticket Classifier

Let's first try K-Nearest Neigbour algorithm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# instantiate the model (with the default parameters)
knn = KNeighborsClassifier()

# fit the model with data (occurs in-place)
knn.fit(trainX_ct, trainY)

Evaluation on Test Dataset

In [ ]:
from sklearn import metrics

In [ ]:
#Calculate accuracy on Test Dataset
predicted_test_y = knn.predict(testX_ct)
metrics.accuracy_score(testY, predicted_test_y)

Classification Report

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
def print_confusion_matrix(testY, predicted_test_y):
    mat = confusion_matrix(testY, predicted_test_y)

    plt.figure(figsize=(4, 4))
    sns.set()
    sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=np.unique(testY),
                yticklabels=np.unique(testY))

    plt.xlabel('true label')
    plt.ylabel('predicted label')
    plt.show()

In [ ]:
print_confusion_matrix(testY, predicted_test_y)

#### We can build Classifier using other algorithms e.g SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
#Train an SVM with default parameters
svc = SVC()
svc.fit(trainX_ct, trainY)

In [ ]:
#Calculate accuracy on Test Dataset
predicted_test_y_svc = svc.predict(testX_ct)
metrics.accuracy_score(testY, predicted_test_y_svc)

In [ ]:
print_confusion_matrix(testY, predicted_test_y_svc)

### 5. Using TF-IDF Vectorizer

In [ ]:
# import and instantiate TF-IDF Vectorizer (with the default parameters)
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(analyzer=get_stemmed_text, min_df=5)

In [ ]:
#Feed data to CountVectorizer
tvect.fit(trainX)

#Check the vocablury size
len(tvect.vocabulary_)

In [ ]:
#Convert Training into numerical values
trainX_tfidf = tvect.transform(trainX)
trainX_tfidf.shape

In [ ]:
#Convert Test SMSes also to tf-idf vectors
testX_tfidf = tvect.transform(testX)

In [ ]:
print(trainX_tfidf[0])

Build an SVM

In [ ]:
svc_tf = SVC()
svc_tf.fit(trainX_tfidf, trainY)

In [ ]:
#Calculate accuracy on Test Dataset
predicted_test_y_svc_tf = svc_tf.predict(testX_tfidf)
metrics.accuracy_score(testY, predicted_test_y_svc_tf)

In [ ]:
print_confusion_matrix(testY, predicted_test_y_svc_tf)

In [ ]:
svc_tf_b = SVC(class_weight='balanced')
svc_tf_b.fit(trainX_tfidf, trainY)

In [ ]:
#Calculate accuracy on Test Dataset
predicted_test_y_svc_tf_b = svc_tf_b.predict(testX_tfidf)
metrics.accuracy_score(testY, predicted_test_y_svc_tf_b)

In [ ]:
print_confusion_matrix(testY, predicted_test_y_svc_tf_b)

TF-IDF with ngram



```
tvect_ngram = TfidfVectorizer(ngram_range=(1,2)) 
#Tokens can be made of 1 word or 2 words
```

The movie was awesome

Words as tokens = "The", "movie", "was", awesome"

ngrams (1,2) -> 
- "The", 
- "movie", 
- "was", 
- "awesome", 
- "The movie", 
- "movie was", 
- "was awesome"

### 7. Building a Deep Learning Model

In [ ]:
import tensorflow as tf

We will use CountVectorizer features in this case. This can be replaced by TF-IDF features

In [ ]:
#Start building a Keras Sequential Model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [ ]:
#Add hidden layers
model.add(tf.keras.layers.Dense(100, activation='relu', input_shape=(len(tvect.vocabulary_),)))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))

#Add Output layer
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.fit(trainX_tfidf, trainY,
           validation_data=(testX_tfidf, testY), 
           epochs=10, batch_size=32)

In [ ]:
#Convert csr sparse matrix to dense
model.fit(trainX_tfidf.todense(), trainY,
           validation_data=(testX_tfidf.todense(), testY), 
           epochs=10, batch_size=32)

In [ ]:
#Calculate accuracy on Test Dataset
predicted_test_y_dense = model.predict(testX_tfidf.todense())

#Convert predictions into binary
predicted_test_y_dense_binary = predicted_test_y_dense >= 0.5

#Calculate score
metrics.accuracy_score(testY, predicted_test_y_dense_binary)

In [ ]:
model.evaluate(testX_tfidf.todense(), testY)

In [ ]:
print_confusion_matrix(testY, predicted_test_y_dense_binary)